# Instrument Classification - Part 1 : Pre-Processing

We start our first trial by trying to recognise piano (pia), acoustic guitar (gac)

In [14]:
import numpy as np
import librosa
import os
import math
from random import shuffle
from collections import deque
from pydub import AudioSegment
from pydub.utils import make_chunks

In [15]:
dirpia = './Instrument-Classification/IRMAS-TrainingData/pia/'
dirgac = './Instrument-Classification/IRMAS-TrainingData/gac/'
#dirsax = './Instrument-Classification/IRMAS-TrainingData/sax/'
#dirvoi = './Instrument-Classification/IRMAS-TrainingData/voi/'

#instr = ['pia','gac','sax','voi']
#dir = [dirpia,dirgac,dirsax,dirvoi]

instr = ['pia','gac']
dir = [dirpia,dirgac]

In [16]:
def pre_processing(dir_array):
    x_list = []
    y_list = []
    list = []
    for dir in dir_array:
        for f in os.listdir(dir):
            list.append(dir+f)
    check = 0
    for f in list:
        label = f.split('/')
        # downsampled to 22050Hz and downmixed to mono
        wav,sr = librosa.load(f,mono=True,sr=22050)
        # normalized by the root mean square energy
        wav = wav / np.sqrt(np.mean(wav**2))
        # chunked to 1sec long snippets
        chunks = make_chunks(wav,sr)
        for wav in chunks:
            # transformed into mel-spectrograms with given traits
            mel = librosa.feature.melspectrogram(wav,n_mels=96,fmax=11025,n_fft=1024,hop_length=256,power=1)
            # decibel scaling
            mel = librosa.core.amplitude_to_db(mel)
            # saved to train and test arrays
                
            #just for checking
            if (check == 500): 
                print("Another 500 chuncks have been made!")
                check=0
            else: 
                check+=1
            y_list.append(label[-2])
            x_list.append(mel)
    return (x_list,y_list)

def to_categorical(list):
    temp_list = [[0]*len(instr)]*(len(list))
    for i in range(len(list)):
        x = [0]*len(instr)
        x.insert(0,1)
        x.pop()
        for ins in instr:
            if (ins in list[i]): 
                temp_list[i] = [sum(x) for x in zip(temp_list[i],x)]
            x.insert(0,x.pop())
    return temp_list

In [17]:
(x_list,y_list) = pre_processing(dir)

x_list = np.expand_dims(np.array(x_list),-1)
y_list = np.array(to_categorical(y_list))

#shuffling sets
id = np.random.permutation(len(x_list))
x_list,y_list = x_list[id], y_list[id]

#splitting to train and test 90/10
split_index = math.floor(len(x_list)*0.90)
x_train = x_list[:split_index]
y_train = y_list[:split_index]
x_test  = x_list[split_index:]
y_test  = y_list[split_index:]

print("Train Size:", x_train.shape)
print("Test  Size:", x_test.shape)

Another 500 chuncks have been made!
Another 500 chuncks have been made!
Another 500 chuncks have been made!
Another 500 chuncks have been made!
Another 500 chuncks have been made!
Another 500 chuncks have been made!
Another 500 chuncks have been made!
Another 500 chuncks have been made!
Another 500 chuncks have been made!
Another 500 chuncks have been made!
Another 500 chuncks have been made!
Another 500 chuncks have been made!
Another 500 chuncks have been made!
Another 500 chuncks have been made!
Another 500 chuncks have been made!
Another 500 chuncks have been made!
Another 500 chuncks have been made!
Another 500 chuncks have been made!
Another 500 chuncks have been made!
Another 500 chuncks have been made!
Another 500 chuncks have been made!
Another 500 chuncks have been made!
Another 500 chuncks have been made!
Another 500 chuncks have been made!
Another 500 chuncks have been made!
Another 500 chuncks have been made!
Another 500 chuncks have been made!
Train Size: (12401, 96, 87, 

In [19]:
np.save('x_trainB.npy',x_train)
np.save('x_testB.npy' ,x_test)
np.save('y_trainB.npy',y_train)
np.save('y_testB.npy' ,y_test)